In [ ]:
from zipfile import ZipFile
file_name = 'test.zip'

with ZipFile(file_name, 'r') as zipped:
  zipped.extractall()
  print('Done')

Done


In [15]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import pickle
import os

In [33]:
def predict_img_svm(img_path, new_img_name, dest_folder):
    # load the model file
    with open('svm_model.pkl', 'rb') as f:
        loaded_model = pickle.load(f)
    # read the image
    img = cv2.imread(img_path)
    # Use orb object to detect and compute descriptors and keypoints
    orb = cv2.ORB_create(scoreType=cv2.ORB_FAST_SCORE, nlevels=1)
    keypoints, descriptors = orb.detectAndCompute(img, None)
    # use the descriptors as the features to predict the label of keypoints
    preds = loaded_model.predict(descriptors)
    # draw keypoints that are predicted to be 1 (in a bbox, or on wheat)
    for kpt, pred in zip(keypoints, preds):
        if pred == 1:
            img = cv2.drawKeypoints(img, [kpt], None, color=(0, 255, 0))
    cv2.imwrite(os.path.join(dest_folder, new_img_name), img)

In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
import torch
import detectron2
from detectron2.engine.defaults import DefaultPredictor
from detectron2.data.transforms import ResizeTransform
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.evaluation import COCOEvaluator
from detectron2.utils.visualizer import Visualizer, ColorMode
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

torch:  2.0 ; cuda:  cu118


In [36]:
def evaluate(image_path, model):
  img = cv2.imread(image_path)
  resize = ResizeTransform(new_h=256, new_w=256, h=1024, w=1024)
  image = resize.apply_image(img)
  return model(img)

def predict_img_dl(file_path, new_img_name, dest_folder):
  cfg = get_cfg()
  cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")) # load the config
  cfg.MODEL.WEIGHTS = 'model_final.pth'  # load the model
  cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 # we only have a class 'wheat'
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold
  MetadataCatalog.get("dataset").thing_classes = ['wheat']
  model = DefaultPredictor(cfg)

  output = evaluate(file_path, model)
  v = Visualizer(cv2.imread(file_path)[:, :, ::-1], metadata=MetadataCatalog.get("dataset"), scale=0.5)

  # Then, we call draw_instance_predictions to overlay our image with the predictions
  v = v.draw_instance_predictions(output["instances"].to("cpu"))
  cv2.imwrite(os.path.join(dest_folder, new_img_name), v.get_image())


In [37]:
if not os.path.exists('svm_output'):
  os.mkdir('svm_output')
if not os.path.exists('dl_output'):
  os.mkdir('dl_output')

test_img_files = os.listdir('test')
test_images = []

for filename in test_img_files:
  filepath = os.path.join('test', filename)
  predict_img_svm(filepath, f'{filename.split(".")[0]}-svm.jpg', 'svm_output')
  predict_img_dl(filepath, f'{filename.split(".")[0]}-dl.jpg', 'dl_output')
